In [ ]:
import pandas as pd

df = pd.read_csv("legal_agreements_dataset_full.csv")

In [ ]:
df.shape

(500, 3)

In [ ]:
df.head()

,Document_ID,Text,Category
0,DOC001,"On March 14, 2025, AlphaGrowth, a Colorado cor...",NDA
1,DOC002,"On July 31, 2025, DeltaTech, a Illinois corpor...",NDA
2,DOC003,"On April 14, 2025, IotaLLC, a Texas corporatio...",NDA
3,DOC004,"On April 11, 2025, KappaTech, a Florida corpor...",NDA
4,DOC005,"On November 04, 2025, IotaLLC, a Florida corpo...",NDA


In [ ]:
# Shuffle rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df.head()

,Document_ID,Text,Category,clean_text
0,DOC362,"Effective June 26, 2025, EpsilonInc, a Califor...",Vendor,"effective june 26, 2025, epsiloninc, a califor..."
1,DOC074,"On May 30, 2025, EtaLabs, a Colorado corporati...",NDA,"on may 30, 2025, etalabs, a colorado corporati..."
2,DOC375,"Effective June 22, 2025, DeltaInc, a Colorado ...",Vendor,"effective june 22, 2025, deltainc, a colorado ..."
3,DOC156,"Effective January 15, 2025, EpsilonTech, a New...",SLA,"effective january 15, 2025, epsilontech, a new..."
4,DOC105,"Effective June 24, 2025, IotaLabs, a Texas cor...",SLA,"effective june 24, 2025, iotalabs, a texas cor..."


# **Cleaning and prprocessing text with Regex (Regular Expression)**

In [ ]:
import re

def clean_text(text):
    # remove special chars except common legal punctuation
    text = re.sub(r"[^a-zA-Z0-9.,;:%\-()\s]", " ", text)
    # normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

df['clean_text'] = df['Text'].apply(clean_text)


# **'Exploring Vectorization with TF-IDF, plus Logistic Regression'**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2), stop_words="english")
X_tfidf = vectorizer.fit_transform(df['clean_text'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

y = df['Category']  # ensure you have a label column with NDA, SLA, etc.

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

  Employment       1.00      1.00      1.00        20
         NDA       1.00      1.00      1.00        15
 Partnership       1.00      1.00      1.00        22
         SLA       1.00      1.00      1.00        21
      Vendor       1.00      1.00      1.00        22

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

[[20  0  0  0  0]
 [ 0 15  0  0  0]
 [ 0  0 22  0  0]
 [ 0  0  0 21  0]
 [ 0  0  0  0 22]]


In [ ]:
df.head(3)

,Document_ID,Text,Category,clean_text
0,DOC362,"Effective June 26, 2025, EpsilonInc, a Califor...",Vendor,"effective june 26, 2025, epsiloninc, a califor..."
1,DOC074,"On May 30, 2025, EtaLabs, a Colorado corporati...",NDA,"on may 30, 2025, etalabs, a colorado corporati..."
2,DOC375,"Effective June 22, 2025, DeltaInc, a Colorado ...",Vendor,"effective june 22, 2025, deltainc, a colorado ..."


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])

# Implementing Logistic Regression with TF-IDF

In [ ]:

X = df["clean_text"]       # column with agreement text
y = df["Category"]      # column with category (NDA, SLA, etc.)

# Split into train & test (stratify keeps class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Vectorize text
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train model
model = LogisticRegression(max_iter=500, class_weight="balanced")
model.fit(X_train_tfidf, y_train)

LogisticRegression(class_weight='balanced', max_iter=500)

In [ ]:
X_test_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 84312 stored elements and shape (100, 4629)>

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# 7. Evaluate
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        20

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



### The lack of randomness in synthetic data presents another challenge of possible Overfitting during training and testing. The model is more likely to memorize data, affecting later predictions

# **Testing Support-Vector Machine**

In [ ]:
from sklearn.svm import LinearSVC

,Text
295,"Effective October 25, 2025, AlphaTech, a Georg..."
346,"On October 22, 2025, EtaGrowth, a Florida corp..."
493,"Effective February 20, 2025, ThetaInc, a Flori..."
177,"Effective September 27, 2025, EpsilonGrowth, a..."
270,"Effective April 18, 2025, DeltaSolutions, a Ne..."
...,...
269,"Effective July 06, 2025, GammaInc, a Florida c..."
92,"Effective February 18, 2025, EtaInnovate, a Ge..."
121,"Effective November 10, 2025, EtaLLC, a New Yor..."
233,"On February 27, 2025, EtaSystems, a Illinois c..."


In [ ]:
svm_model = LinearSVC()
svm_model.fit(X_train, y_train)

LinearSVC()

In [ ]:
y_pred = svm_model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

  Employment       1.00      1.00      1.00        20
         NDA       1.00      1.00      1.00        15
 Partnership       1.00      1.00      1.00        22
         SLA       1.00      1.00      1.00        21
      Vendor       1.00      1.00      1.00        22

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

[[20  0  0  0  0]
 [ 0 15  0  0  0]
 [ 0  0 22  0  0]
 [ 0  0  0 21  0]
 [ 0  0  0  0 22]]


# **XGBOOST**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv("legal_agreements_dataset_full.csv")

# Shuffle rows (keeps text + labels aligned)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Features and labels
X = df["Text"]       # column with agreement text
y = df["Category"]      # column with category (NDA, SLA, etc.)

# Label encode the target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)


X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# Vectorizing with TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Training the XGBoost classifier
model = XGBClassifier(
    objective="multi:softmax",
    num_class=len(le.classes_),  # number of categories
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state=42,
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8
)

model.fit(X_train_tfidf, y_train_encoded)


y_pred_encoded = model.predict(X_test_tfidf)

# Decode the predicted labels back to original category names for the classification report
y_pred = le.inverse_transform(y_pred_encoded)


print("Accuracy:", accuracy_score(y_test_encoded, y_pred_encoded))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [03:33:21] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        20

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



# **LegalBert**

In [ ]:
pip install transformers datasets torch scikit-learn

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


df = pd.read_csv("legal_agreements_dataset_full.csv")

# Shuffle to improve randomness
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Features & labels
X = df["Text"].tolist()
y = df["Category"].astype("category")
label2id = {label: i for i, label in enumerate(y.cat.categories)}
id2label = {i: label for label, i in label2id.items()}
y_ids = y.map(label2id).tolist()

# Test size set to 20%
train_texts, test_texts, train_labels, test_test_labels = train_test_split(
    X, y_ids, test_size=0.2, stratify=y, random_state=42
)

train_df = pd.DataFrame({"Text": train_texts, "labels": train_labels})
test_df = pd.DataFrame({"Text": test_texts, "labels": test_test_labels})

# Convert to HuggingFace dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load the LegalBERT tokenizer-AutoTokenizer and model
model_name = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# Apply tokenization function to both our training and test set
def tokenize(batch):
    return tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set dataset format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Performance metrics, how does legalbert do on this data
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


trainer.train()

# Compute final evaluation
metrics = trainer.evaluate()
print(metrics)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tettehmaxwell11 (tettehmaxwell11-boulder-research) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.169432,1.000000,1.000000,1.000000,1.000000
2,No log,0.035914,1.000000,1.000000,1.000000,1.000000
3,No log,0.024785,1.000000,1.000000,1.000000,1.000000


{'eval_loss': 0.02478545904159546, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 2.7785, 'eval_samples_per_second': 35.991, 'eval_steps_per_second': 4.679, 'epoch': 3.0}


# **Zero-shot Learning: Much preferred Transformers framework**

In [ ]:
from transformers import pipeline

# Load a zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Candidate labels (your legal categories)
labels = ["NDA", "SLA", "Employment", "Vendor", "Partnership"]

# Example texts (you can replace with rows from your dataset)
texts = [
    "This agreement ensures that confidential information will not be disclosed to third parties.",
    "The service provider agrees to maintain 99.9% uptime in this agreement.",
    "This contract confirms the employment terms including salary and benefits.",
    "Vendor shall deliver the goods within 30 days of purchase order.",
    "The two parties agree to collaborate on a joint venture project."
]

for text in texts:
    result = classifier(text, labels)
    print("\nText:", text)
    print("Prediction:", result["labels"][0], " (score:", result["scores"][0], ")")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0



Text: This agreement ensures that confidential information will not be disclosed to third parties.
Prediction: NDA  (score: 0.3385591506958008 )

Text: The service provider agrees to maintain 99.9% uptime in this agreement.
Prediction: Partnership  (score: 0.23931586742401123 )

Text: This contract confirms the employment terms including salary and benefits.
Prediction: Employment  (score: 0.6685754656791687 )

Text: Vendor shall deliver the goods within 30 days of purchase order.
Prediction: Vendor  (score: 0.7394172549247742 )

Text: The two parties agree to collaborate on a joint venture project.
Prediction: Partnership  (score: 0.47802209854125977 )


In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# Load your dataset
df = pd.read_csv("legal_agreements_dataset_full.csv")

# Candidate labels (better as natural language for zero-shot)
labels = ["Non-Disclosure Agreement", "Service Level Agreement",
          "Employment Contract", "Vendor Agreement", "Partnership Agreement"]

# Load the zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Create a list to store predictions
predicted_labels = []

# tqdm gives you a progress bar so you know how long it takes
for text in tqdm(df["Text"], desc="Classifying"):
    result = classifier(text, labels)
    # take the label with the highest score
    predicted_labels.append(result["labels"][0])

# Add predictions to new column
df["Predicted_Label"] = predicted_labels

df.head()


Device set to use cuda:0
Classifying: 100%|██████████| 500/500 [08:10<00:00,  1.02it/s]


,Document_ID,Text,Category,Predicted_Label
0,DOC001,"On March 14, 2025, AlphaGrowth, a Colorado cor...",NDA,Partnership Agreement
1,DOC002,"On July 31, 2025, DeltaTech, a Illinois corpor...",NDA,Partnership Agreement
2,DOC003,"On April 14, 2025, IotaLLC, a Texas corporatio...",NDA,Partnership Agreement
3,DOC004,"On April 11, 2025, KappaTech, a Florida corpor...",NDA,Partnership Agreement
4,DOC005,"On November 04, 2025, IotaLLC, a Florida corpo...",NDA,Partnership Agreement


In [ ]:
# Create a mapping from the zero-shot predicted labels to the original category labels
label_mapping = {
    "Partnership Agreement": "Partnership",
    "Non-Disclosure Agreement": "NDA",
    "Service Level Agreement": "SLA",
    "Employment Contract": "Employment",
    "Vendor Agreement": "Vendor"
}

# Apply the mapping to the 'Predicted_Label' column
df["Predicted_Label"] = df["Predicted_Label"].map(label_mapping)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

y_true = df["Category"]
y_pred = df["Predicted_Label"]

# Generate classification report
report = classification_report(y_true, y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

  Employment       1.00      0.79      0.88       100
         NDA       0.00      0.00      0.00       100
 Partnership       0.28      1.00      0.44       100
         SLA       1.00      0.04      0.08       100
      Vendor       0.00      0.00      0.00       100

    accuracy                           0.37       500
   macro avg       0.46      0.37      0.28       500
weighted avg       0.46      0.37      0.28       500



In [ ]:
import pandas as pd
df=pd.read_csv('/content/legal_agreements_dataset_full.csv')

In [ ]:
df[df['Category']=='NDA'].iloc[3,1]

"On April 11, 2025, KappaTech, a Florida corporation at 455 Privacy Lane, Orlando, Florida, and AlphaLabs, a Colorado limited liability company at 749 Gear Street, Boulder, Colorado, enter a covenant to protect sensitive information during merger talks. The terms include: 1. Protected Data Definition. 'Confidential Information' includes business plans, financial projections, source code. 2. Recipient Duties. The receiving Party restricts access to authorized personnel and uses data only for merger talks. 3. Exclusions. Obligations exclude data publicly known, previously possessed, or independently developed. 4. Duration. The covenant lasts 2 years, with data return or destruction upon termination. 5. Jurisdiction. Florida law governs, with disputes in Orlando courts. The Parties execute this covenant to safeguard discussions. Additionally, access logs for Confidential Information are maintained and provided upon request. Reverse engineering or decompiling disclosed materials is prohibi

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "valhalla/distilbart-mnli-12-1"
save_path = "./distilbart-mnli/"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/890M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/890M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

MODEL_PATH = "./distilbart-mnli/"

# Load the model + tokenizer from your saved directory
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer)

# Test with a sample input
text = "This agreement is made between the employer and the employee regarding workplace policies."
labels = ["Employment", "Vendor", "NDA", "Partnership", "SLA"]

result = classifier(text, candidate_labels=labels)

print(result)


Device set to use cuda:0


{'sequence': 'This agreement is made between the employer and the employee regarding workplace policies.', 'labels': ['Employment', 'NDA', 'Partnership', 'SLA', 'Vendor'], 'scores': [0.4167461097240448, 0.27939122915267944, 0.2395477443933487, 0.05437755584716797, 0.009937391616404057]}


In [ ]:
import shutil

shutil.make_archive("distilbart-mnli", 'zip', "./distilbart-mnli")

'/content/distilbart-mnli.zip'

In [ ]:
from google.colab import files
files.download("distilbart-mnli.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>